In [1]:
from imports import *

In [2]:
galaxy = 'ngc1672'
galaxy_muse = galaxy
rootdir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/%s/' %galaxy
rootdir_bp = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/hst_filters/' 

narrowband_filter = 'f658n'
instrument_f555w  = 'acs'
instrument_f65Xn = 'acs'
instrument_f814w = 'acs'

# Load files
hdu_muse_f555w  = get_hdu(rootdir, 'muse/%s*_%s_*F550M*.fits' %(galaxy_muse.upper(), instrument_f555w.upper()), hdu_id=1)
hdu_muse_f65Xn  = get_hdu(rootdir, 'muse/%s*_%s_*%s.fits' %(galaxy_muse.upper(), instrument_f65Xn.upper(), narrowband_filter.upper()))
hdu_muse_f814w  = get_hdu(rootdir, 'muse/%s*_%s_*F814W.fits' %(galaxy_muse.upper(), instrument_f814w.upper()))

hdu_muse_stars  = get_hdu(rootdir, 'muse/%s_starmask.fits' %galaxy_muse.upper())
hdu_muse_neb    = get_hdu(rootdir, 'muse/%s_nebmask.fits' %galaxy_muse.upper())
hdu_muse        = get_hdu(rootdir, 'muse/%s*_MAPS.fits' %galaxy_muse.upper(), 'all')

hdu_hst_f555w   = get_hdu(rootdir, 'hst/%s*_%s_*f550m*_exp_drc_sci.fits' %(galaxy, instrument_f555w))
hdu_hst_f65Xn   = get_hdu(rootdir, 'hst/%s*_%s_*%s*_exp_drc_sci.fits' %(galaxy, instrument_f65Xn, narrowband_filter))
hdu_hst_f814w   = get_hdu(rootdir, 'hst/%s*_%s_*f814w*_exp_drc_sci.fits' %(galaxy, instrument_f814w))

# Get Av for MW extinction correction
Av = get_Av(galaxy_muse)

# Get resolution 
hst_res  = 0.07 * u.arcsec
_, file_muse_f65Xn = get_hdu(rootdir, 'muse/%s*_%s_*%s.fits' %(galaxy_muse.upper(), instrument_f65Xn.upper(), narrowband_filter.upper()), return_filename=True)
muse_res = np.float32(file_muse_f65Xn.split('asec')[0].split('-')[-1]) * u.arcsec

# Get filter curve info
bp = get_bandpassinfo(rootdir_bp)

# Cosmic ray thresholds
cr_threshold = 0.9
cr_dilation_iterations = 0

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1672/muse/NGC1672-0.96asec_ACS_F550M_v2.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1672/muse/NGC1672-0.96asec_ACS_F658N.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1672/muse/NGC1672-0.96asec_ACS_F814W.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1672/muse/NGC1672_starmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1672/muse/NGC1672_nebmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1672/muse/NGC1672-0.96asec_MAPS.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1672/hst/ngc1672_acs_f550m_exp_drc_sci.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1672/hst/ngc1672_acs_f658n_exp_drc_sci.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1672/hst/ngc1672_acs_f814w_exp_drc_sci.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/dat

In [3]:
do_main = True
if do_main: 
    
    # Make paths
    make_paths(rootdir, appdir='hst_contsub_f550m/')
    remove_paths(rootdir, appdir='hst_contsub_f550m/')

    # Convert units 
    hdu_hst_f555w = get_nanzeros(hdu_hst_f555w)
    hdu_hst_f65Xn = get_nanzeros(hdu_hst_f65Xn)
    hdu_hst_f814w = get_nanzeros(hdu_hst_f814w)

    hdu_hst_f65Xn = remove_nan_padding(hdu_hst_f65Xn)
    hdu_hst_f555w = get_regrid(hdu_hst_f555w, hdu_hst_f65Xn)
    hdu_hst_f814w = get_regrid(hdu_hst_f814w, hdu_hst_f65Xn)

    hdu_hst_f555w, hdu_hst_f65Xn, hdu_hst_f814w = get_covmask(hdu_hst_f555w, hdu_hst_f65Xn, hdu_hst_f814w)

    hdu_hst_f555w = get_electrons_2_ergcm2sA(hdu_hst_f555w)
    hdu_hst_f65Xn = get_electrons_2_ergcm2sA(hdu_hst_f65Xn)
    hdu_hst_f814w = get_electrons_2_ergcm2sA(hdu_hst_f814w)

    hdu_muse_f65Xn = get_Jy_2_ergcm2sA(hdu_muse_f65Xn, bp['%s_%s' %(instrument_f65Xn.upper(), narrowband_filter.upper())]['pivot'])
    hdu_muse_f814w = get_Jy_2_ergcm2sA(hdu_muse_f814w, bp['%s_%s' %(instrument_f814w.upper(), 'F814W')]['pivot'])

    # Smooth and regrid to MUSE
    hdu_hst_f555w_sm = get_smooth(hdu_hst_f555w, hst_res, muse_res)
    hdu_hst_f65Xn_sm = get_smooth(hdu_hst_f65Xn, hst_res, muse_res)
    hdu_hst_f814w_sm = get_smooth(hdu_hst_f814w, hst_res, muse_res)

    hdu_hst_f555w_smre = get_regrid(hdu_hst_f555w_sm, hdu_muse_f555w)
    hdu_hst_f65Xn_smre = get_regrid(hdu_hst_f65Xn_sm, hdu_muse_f65Xn)
    hdu_hst_f814w_smre = get_regrid(hdu_hst_f814w_sm, hdu_muse_f814w)

    # Anchor filter flux density to MUSE
    hdu_hst_f555w_an, hdu_hst_f555w_smrean, fit_f555w = get_anchoring_offset(hdu_muse_f555w, hdu_hst_f555w_smre, hdu_hst_f555w, hdu_muse_stars, 'f550m', rootdir)
    hdu_hst_f65Xn_an, hdu_hst_f65Xn_smrean, fit_f65Xn = get_anchoring_offset(hdu_muse_f65Xn, hdu_hst_f65Xn_smre, hdu_hst_f65Xn, hdu_muse_stars, narrowband_filter, rootdir)
    hdu_hst_f814w_an, hdu_hst_f814w_smrean, fit_f814w = get_anchoring_offset(hdu_muse_f814w, hdu_hst_f814w_smre, hdu_hst_f814w, hdu_muse_stars, 'f814w', rootdir)

    # Correct for MW extinction
    hdu_hst_f555w.data = correct_extinction(hdu_hst_f555w.data, Av, inst=instrument_f555w.upper(), filt='F550M')
    hdu_hst_f65Xn.data = correct_extinction(hdu_hst_f65Xn.data, Av, inst=instrument_f65Xn.upper(), filt=narrowband_filter.upper())
    hdu_hst_f814w.data = correct_extinction(hdu_hst_f814w.data, Av, inst=instrument_f814w.upper(), filt='F814W')

    hdu_muse_f555w.data = correct_extinction(hdu_muse_f555w.data, Av, inst=instrument_f555w.upper(), filt='F550M')
    hdu_muse_f65Xn.data = correct_extinction(hdu_muse_f65Xn.data, Av, inst=instrument_f65Xn.upper(), filt=narrowband_filter.upper())
    hdu_muse_f814w.data = correct_extinction(hdu_muse_f814w.data, Av, inst=instrument_f814w.upper(), filt='F814W')

    hdu_hst_f555w_an.data = correct_extinction(hdu_hst_f555w_an.data, Av, inst=instrument_f555w.upper(), filt='F550M')
    hdu_hst_f65Xn_an.data = correct_extinction(hdu_hst_f65Xn_an.data, Av, inst=instrument_f65Xn.upper(), filt=narrowband_filter.upper())
    hdu_hst_f814w_an.data = correct_extinction(hdu_hst_f814w_an.data, Av, inst=instrument_f814w.upper(), filt='F814W')

    # Continuum subtract 
    output_hst = get_contsub(hdu_hst_f65Xn.copy(), 
                                    hdu_hst_f555w.copy(), 
                                    hdu_hst_f814w.copy(), 
                                    bp['%s_%s' %(instrument_f65Xn.upper(), narrowband_filter.upper())]['pivot'], 
                                    bp['%s_%s' %(instrument_f555w.upper(), 'F550M')]['pivot'], 
                                    bp['%s_%s' %(instrument_f814w.upper(), 'F814W')]['pivot'])
    hdu_hst_halpha, hdu_hst_cont = output_hst

    output_hst_an = get_contsub(hdu_hst_f65Xn_an.copy(), 
                                    hdu_hst_f555w_an.copy(), 
                                    hdu_hst_f814w_an.copy(), 
                                    bp['%s_%s' %(instrument_f65Xn.upper(), narrowband_filter.upper())]['pivot'], 
                                    bp['%s_%s' %(instrument_f555w.upper(), 'F550M')]['pivot'], 
                                    bp['%s_%s' %(instrument_f814w.upper(), 'F814W')]['pivot'])       
    hdu_hst_an_halpha, hdu_hst_an_cont = output_hst_an

    output_muse = get_contsub(hdu_muse_f65Xn.copy(), 
                                hdu_muse_f555w.copy(), 
                                hdu_muse_f814w.copy(), 
                                bp['%s_%s' %(instrument_f65Xn.upper(), narrowband_filter.upper())]['pivot'], 
                                bp['%s_%s' %(instrument_f555w.upper(), 'F550M')]['pivot'], 
                                bp['%s_%s' %(instrument_f814w.upper(), 'F814W')]['pivot'])  
    hdu_muse_halpha, hdu_muse_cont = output_muse

    # Convert units
    photbw = bp['%s_%s' %(instrument_f65Xn.upper(), narrowband_filter.upper())]['rectwidth']

    hdu_hst_halpha        = get_ergcm2sA_2_ergcm2s(hdu_hst_halpha, photbw)
    hdu_hst_cont          = get_ergcm2sA_2_ergcm2s(hdu_hst_cont, photbw)
    hdu_hst_an_halpha     = get_ergcm2sA_2_ergcm2s(hdu_hst_an_halpha, photbw)
    hdu_hst_an_cont       = get_ergcm2sA_2_ergcm2s(hdu_hst_an_cont, photbw)
    hdu_muse_halpha       = get_ergcm2sA_2_ergcm2s(hdu_muse_halpha, photbw)
    hdu_muse_cont         = get_ergcm2sA_2_ergcm2s(hdu_muse_cont, photbw)
    hdu_hst_f65Xn_anf     = get_ergcm2sA_2_ergcm2s(hdu_hst_f65Xn_an, photbw)

    # Anchor Ha flux to MUSE
    hdu_hst_halpha_scaled, hdu_muse_halpha_scaled, fit_halpha = get_anchoring_slope(hdu_muse['HA6562_FLUX'], hdu_muse_halpha, hdu_hst_halpha, hdu_muse_stars, '', rootdir, make_plots=False)
    hdu_hst_an_halpha_scaled, hdu_muse_an_halpha_scaled, fit_an_halpha = get_anchoring_slope(hdu_muse['HA6562_FLUX'], hdu_muse_halpha, hdu_hst_an_halpha, hdu_muse_stars, '', rootdir, make_plots=False)

    # Save files
    write_hdu(hdu_hst_an_halpha_scaled, rootdir, '%s_hst_ha.fits' %galaxy, appdir='hst_contsub_f550m/')

[Info] Outputing to the following: /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1672/hst_contsub_f550m/
[INFO] Remove NaN values around edge of image...
[INFO] Reprojecting the input image to match the template WCS...
[INFO] Performing image reprojection...
[INFO] Image reprojection complete.
[INFO] Scaling the output data to conserve flux with factor 1.00
[INFO] Flux scaling complete.
[INFO] Reprojection process completed.
[INFO] Reprojecting the input image to match the template WCS...
[INFO] Performing image reprojection...
[INFO] Image reprojection complete.
[INFO] Scaling the output data to conserve flux with factor 1.00
[INFO] Flux scaling complete.
[INFO] Reprojection process completed.
[INFO] Pixel scale: 0.04 arcsec arcsec
[INFO] Initial Resolution: 0.07 arcsec arcsec
[INFO] Desired Resolution: 0.96 arcsec arcsec
[INFO] Convolution kernel: 0.96 arcsec arcsec
[INFO] Performing image convolution...
[INFO] Image convolution complete.
[INFO] Smoothing process c